# Chapter 4 Lab 3

## Goal

The goal of this lab is to demonstrate how you can use Python to perform feature selection - specifically forward and backward selection.

First, we read in the normalized numeric data from Lab 1 of this chapter.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from yellowbrick.cluster import KElbowVisualizer

In [2]:
dota_df1 = pd.read_csv("Dota_normalized.csv")
dota_df1.head()

,GamesPlayed,GamesWon,GamesLeft,Ditches,Points,Kills,KillsPerMin,Deaths,Assists,CreepsKilled,CreepsDenied,NeutralsKilled,TowersDestroyed,RaxsDestroyed,TotalTime
0,0.141363,0.140590,0.150,0.333333,0.350240,0.095653,0.285714,0.111196,0.122158,0.104007,0.069642,0.080559,0.136486,0.132340,0.259791
1,0.020602,0.022109,0.000,0.000000,0.262429,0.015711,0.309524,0.026489,0.021663,0.012018,0.011797,0.010606,0.018839,0.020158,0.039233
2,0.000634,0.000000,0.000,0.000000,0.269743,0.000463,0.261905,0.001648,0.000523,0.000475,0.000228,0.000163,0.000000,0.000000,0.001125
3,0.031379,0.033447,0.125,0.055556,0.434886,0.045447,0.619048,0.026372,0.027159,0.037422,0.042929,0.033030,0.050750,0.031551,0.057959
4,0.000000,0.000567,0.000,0.000000,0.284200,0.000379,0.666667,0.000118,0.000291,0.000078,0.000182,0.000070,0.000000,0.000000,0.000139
